In [1]:
import os
import sys
import pandas as pd
import numpy as np
import PIL

seed = 16
np.random.seed(seed)

from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator


c:\users\garrick\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#check using system GPU for processing

from tensorflow.python.client import device_lib
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for training on gpu
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1583412771044383216
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1493781708
locality {
  bus_id: 1
}
incarnation: 15482032816628967102
physical_device_desc: "device: 0, name: GeForce GTX 860M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [3]:
# declare ImageDataGenerator and flow_from_directory vars
batch_size=10
num_classes = 120

In [4]:
# changing directory for flow_from_directory method
os.chdir('C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets')

In [5]:
#obtain train and test labels
from scipy.io import loadmat

y_train_rough = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']

y_test = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\test_list.mat''')['labels']

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
labels = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['labels']
files = loadmat(r'''C:\\Users\\Garrick\\Documents\\Springboard\\Capstone Project 2\\datasets\\train_list.mat''')['file_list']
labels = [item for label in labels for item in label] #this is flattening a list of lists, because for some reason ever label is stored as a list
files = [item for file in files for item in file]
df = pd.DataFrame({'labels':labels, 'files':files})
train, validate = train_test_split(df, test_size = 0.2, stratify=labels)


In [8]:
X_train = train['files']
y_train = to_categorical(train['labels'])
X_val = validate['files']
y_val = to_categorical(validate['labels'])

In [9]:
train_datagen = ImageDataGenerator(rotation_range=15, shear_range=0.1, channel_shift_range=20,
                                    width_shift_range=0.1,  height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True,
                                    fill_mode='nearest', rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)



train_generator = train_datagen.flow_from_directory('train', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)

validation_generator = validation_datagen.flow_from_directory('validation', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


test_generator = test_datagen.flow_from_directory('test', target_size=(224,224), color_mode='rgb',
            class_mode='categorical', shuffle=False, batch_size=batch_size)


Found 9600 images belonging to 120 classes.
Found 2400 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [24]:
#obtain validation labels
# no longer used, utilizing sklearn's train_test_split function

'''
y_validation = []
y_train = []

for i in range(120):
    i = i*100
    
    #begin_index = train_full_list[i::100]
    #end_index = train_full_list[i+100::100]
    
    slice = y_train_rough[i:i+100,]
    y_train.append(slice[:80])
    y_validation.append(slice[80:])

y_train = np.concatenate(y_train, axis=0)
y_validation = np.concatenate(y_validation, axis=0)
'''

'\ny_validation = []\ny_train = []\n\nfor i in range(120):\n    i = i*100\n    \n    #begin_index = train_full_list[i::100]\n    #end_index = train_full_list[i+100::100]\n    \n    slice = y_train_rough[i:i+100,]\n    y_train.append(slice[:80])\n    y_validation.append(slice[80:])\n\ny_train = np.concatenate(y_train, axis=0)\ny_validation = np.concatenate(y_validation, axis=0)\n'

In [10]:
# using a simple CNN to start

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import keras.utils
from keras import backend as K
K.set_image_dim_ordering('tf')

from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2)

In [11]:
input_shape = (224,224, 3)

# create the model

model = Sequential()
model.add(Conv2D(64, (11, 11), strides=4, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (4, 4), strides=2, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
#model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
     

# Compile model
epochs = 10
lrate = 0.003
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 64)        23296     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        65600     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 2, 2, 64)          65600     
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
__________

In [13]:
# configure tensorflow before fitting model
tf_config = tf.ConfigProto()
tf_config.gpu_options(per_process_gpu_memory_fraction = 0.99, allow_growth=True)
sess = tf.Session(config=tf_config)

In [14]:
# train/fit the simple CNN using flow from directory 

model.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=len(X_train) / 32, epochs=epochs, callbacks=[early_stopping])

Epoch 1/10
300/300 [==============================] - 124s 414ms/step - loss: 4.8015 - acc: 0.0037 - val_loss: 4.7891 - val_acc: 0.0083
Epoch 2/10
300/300 [==============================] - 89s 297ms/step - loss: 4.8049 - acc: 0.0030 - val_loss: 4.7885 - val_acc: 0.0083
Epoch 3/10
300/300 [==============================] - 103s 345ms/step - loss: 4.8099 - acc: 0.0000e+00 - val_loss: 4.7877 - val_acc: 0.0083
Epoch 4/10
300/300 [==============================] - 76s 255ms/step - loss: 4.7930 - acc: 0.0087 - val_loss: 4.7880 - val_acc: 0.0083
Epoch 5/10
300/300 [==============================] - 68s 226ms/step - loss: 4.7972 - acc: 0.0000e+00 - val_loss: 4.7881 - val_acc: 0.0083


In [15]:
# trying model with more trainable parameters

model_2 = Sequential()
model_2.add(Conv2D(64, (4, 4), strides=2, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_2.add(MaxPooling2D(pool_size=(2, 2)))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes, activation='softmax'))
     

# Compile model
epochs = 10
lrate = 0.003
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model_2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model_2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 111, 111, 64)      3136      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 53, 53, 32)        18464     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
__________

In [24]:
#trying for a a longer epoch *** DONT RUN, accuracy steadily increases but still low ACC

model_2.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=9600, epochs=epochs, callbacks=[early_stopping])

Epoch 1/25
9600/9600 [==============================] - 2010s 209ms/step - loss: 4.7933 - acc: 0.0036 - val_loss: 4.7876 - val_acc: 0.0083
Epoch 2/25
9600/9600 [==============================] - 2051s 214ms/step - loss: 4.7901 - acc: 0.0041 - val_loss: 4.7875 - val_acc: 0.0088
Epoch 3/25
7192/9600 [=====================>........] - ETA: 8:39 - loss: 4.7893 - acc: 0.0049

KeyboardInterrupt: 

In [20]:
# trying no dropout, diff optimizer
from keras.optimizers import Adam

model_3 = Sequential()
model_3.add(Conv2D(64, (4, 4), strides=2, input_shape=input_shape, padding='valid', activation='relu', kernel_constraint=maxnorm(3)))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(64, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Conv2D(32, (3, 3), strides=1, activation='relu', padding='valid', kernel_constraint=maxnorm(3)))
model_3.add(MaxPooling2D(pool_size=(2, 2)))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
model_3.add(Dropout(0.25))
model_3.add(Dense(num_classes, activation='softmax'))
     

# Compile model
epochs = 25
adam_op = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_3.compile(loss='categorical_crossentropy', optimizer=adam_op, metrics=['accuracy'])
print(model_3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 111, 111, 64)      3136      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 53, 53, 64)        36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 24, 24, 32)        18464     
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 4608)              0         
__________

In [21]:
model_3.fit_generator(train_generator, validation_data=validation_generator,
                    steps_per_epoch=len(X_train) / 32, epochs=epochs, callbacks=[early_stopping])

Epoch 1/25
300/300 [==============================] - 70s 234ms/step - loss: 4.7972 - acc: 0.0073 - val_loss: 4.7872 - val_acc: 0.0104
Epoch 2/25
300/300 [==============================] - 74s 248ms/step - loss: 4.7902 - acc: 0.0017 - val_loss: 4.7875 - val_acc: 0.0088
Epoch 3/25
300/300 [==============================] - 71s 235ms/step - loss: 4.7907 - acc: 0.0037 - val_loss: 4.7874 - val_acc: 0.0092


In [ ]:
from keras.applications import VGG16
 
#Load the VGG model
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=input_shape) 

In [ ]:
ImageDataGenerator?